In [1]:
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np

PLEASE UPLOAD THE FILES FROM DRIVE TO COLLAB BEFORE RUNNING THE CODE! Data taken from https://raw.githubusercontent.com/zpettry/AI-Deep-Learning-for-Phishing-URL-Detection/master/whitelist.txt
https://raw.githubusercontent.com/zpettry/AI-Deep-Learning-for-Phishing-URL-Detection/master/phishing_database.csv

In [2]:
    blacklist = 'phishing_database.csv'
    whitelist = 'whitelist.txt'

    urls = {} #array for urls
    
    blacklist = pd.read_csv(blacklist) # read file with malicious URLs

    #Assign 1 as malicious for supervised learning.
    for url in blacklist['url']:
        urls[url] = 1
    # Read file with safe URLs and Assign 0 as safe for supervised learning. 
    with open(whitelist, 'r') as f:
        lines = f.read().splitlines()
        for url in lines:
            urls[url] = 0



In [3]:
samples = [] 
labels = [] #arrays samples and labels were created to store url and their safety score

for k, v in urls.items():
    samples.append(k)
    labels.append(v)
    
#tokenising and finding unique tokens to increase accuracy

max_chars = 5000 
maxlen = 256

#Tokenising data
tokenizer = Tokenizer(num_words=max_chars, char_level=True) 
tokenizer.fit_on_texts(samples) 

#Tokens are sequenced
sequences = tokenizer.texts_to_sequences(samples) 

data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)


In [4]:
#dividing samples
training_samples = int(len(samples) * 0.9)
validation_samples = int(len(labels) * 0.1)


In [5]:
#shuffling data and spliting into test and fit
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x = data[:training_samples]
y = labels[:training_samples]
x_test = data[training_samples: training_samples + validation_samples]
y_test = labels[training_samples: training_samples + validation_samples]



In [6]:
#Logistics regression, avg scor: 66%
model1 = LogisticRegression(max_iter=69420)
model1.fit(x, y)
model1.score(x_test,y_test)


0.7144578313253012

In [7]:
#K neighbouts, average score 88%
model2=KNeighborsClassifier()
model2.fit(x,y)
model2.score(x_test,y_test)


0.889328743545611

In [8]:
# DecisionTrees, average score 86%
model3=DecisionTreeClassifier()
model3.fit(x,y)
model3.score(x_test,y_test)

0.8740103270223752

In [9]:
yout2=model2.predict(x_test)
yout3=model3.predict(x_test)

In [10]:
#Final decision making process combining the 2 algorithms: if at leas one detects phishing url (1), url is treated as phishing

combined_correct=0
combined_false_pos_pos=0
combined_false_neg_neg=0
combined_false_pos=0
for i in range(len(x_test)):
  if (yout2[i]==0 and yout3[i]==0 and y_test[i]==0) or (yout2[i]==1 and yout3[i]==1 and y_test[i]==1) or (yout2[i]==1 and yout3[i]==0 and y_test[i]==1) or (yout2[i]==0 and yout3[i]==1 and y_test[i]==1) :
    combined_correct+=1
  if yout2[i]==1 and yout3[i]==1 and y_test[i]==0:
    combined_false_pos_pos+=1
  if yout2[i]==0 and yout3[i]==0 and y_test[i]==1:
    combined_false_neg_neg+=1
  if (yout2[i]==0 and yout3[i]==1 and y_test[i]==0) or (yout2[i]==1 and yout3[i]==0 and y_test[i]==0):
    combined_false_pos+=1


print("Accuracy:" + str(combined_correct/len(x_test)*100) + "%", "False positives:" + str((combined_false_pos_pos+combined_false_pos)/len(x_test)*100) + "%", "False negatives:" + str((combined_false_neg_neg/len(x_test))*100) + "%" )


Accuracy:90.51635111876077% False positives:6.660929432013769% False negatives:2.8227194492254735%
